In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http:

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-09 10:39:31--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.72MB/s    in 0.2s    

2022-09-09 10:39:32 (5.72 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in amazon luggage.tsv from S3 into a DataFrame
url = "https://myawsbuckethw22.s3.us-west-1.amazonaws.com/amazon_reviews_us_PC_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...|2015-01-31 00:08:00|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...| 

In [5]:
print(df.count())

6908554


In [6]:
customer_id_df = df.select("customer_id")
customer_id_df.show()

+-----------+
|customer_id|
+-----------+
|   22873041|
|   30088427|
|   20329786|
|   14215710|
|   38264512|
|   30548466|
|     589298|
|   49329488|
|   50728290|
|   37802374|
|   52027882|
|   41770239|
|   42560427|
|   46345923|
|   41751192|
|   21176481|
|   10674058|
|   43341796|
|   13232866|
|   29333557|
+-----------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3ARRMDEGED8RD|   22873041|B00KJWQIIC|     335625766| 2015-01-31|
| RQ28TSA020Y6J|   30088427|B013ALA9LA|     671157305| 2015-01-31|
| RUXJRZCT6953M|   20329786|B00PML2GQ8|     982036237| 2015-01-31|
| R7EO0UO6BPB71|   14215710|B001NS0OZ4|     576587596| 2015-01-31|
|R39NJY2YJ1JFSV|   38264512|B00AQMTND2|     964759214| 2015-01-31|
|R31SR7REWNX7CF|   30548466|B00KX4TORI|     170101802| 2015-01-31|
| RVBP8I1R0CTZ8|     589298|B00P17WEMY|     206124740| 2015-01-31|
|R1QF6RS1PDLU18|   49329488|B00TR05L9Y|     778403103| 2015-01-31|
|R23AICGEDAJQL1|   50728290|B0098Y77OG|     177098042| 2015-01-31|
|R2EY3N4K9W19UP|   37802374|B00IFYEYXC|     602496520| 2015-01-31|
| RC9AW4HKJ016M|   52027882|B0091ITP0S|     977217357| 2015-01-31|
|R2ALWJE9N6ZBXD|   41770239|B008I21EA2|     295632907| 2015-01

In [8]:
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0016P392O|iMicro SP-IM942 M...|
|B00PPRYEZA|CoulVue Holder an...|
|B00VZXRRGG|Hexagon Network -...|
|B007GF9N96|Computer USB-Powe...|
|B00LCK3TEG|           Fire HD 6|
|B00KMOG5FS|Fintie 10.1" Tabl...|
|B00CFL5CRI|Cable Matters 2-P...|
|B008NP2AUC|USB USB-2, USB-3 ...|
|B00OAJ5N6I|Samsung 850 EVO -...|
|B00H02XN2I|Techno Earth Rubb...|
|B00YSC6UW8|5 Mm Thick Office...|
|B00CRXZPC6|Lenovo Ideatab S6...|
|B009SKTZFY|Bosign Tabletpill...|
|B003NZSX7G|Netgear N300 WiFi...|
|B0089JIDLC|Kingston ValueRAM...|
|B001E45XT4|Plantronics GameC...|
|B002IY4V9C|Swiss Gear Men's ...|
|B000UVRU6G|SteelSeries QcK G...|
|B00X5X3LQY|EVGA Pro SLI Brid...|
|B00L49X8E6|ASUS X551 15.6-in...|
+----------+--------------------+
only showing top 20 rows



In [9]:
reviews_df = df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R3ARRMDEGED8RD|Pleasantly surprised|I was very surpri...|
| RQ28TSA020Y6J|OnHub is a pretty...|I am a Google emp...|
| RUXJRZCT6953M|None of them work...|Bought cables in ...|
| R7EO0UO6BPB71|just keep searching.|nope, cheap and slow|
|R39NJY2YJ1JFSV|          Five Stars|Excellent! Great ...|
|R31SR7REWNX7CF|Good quality, wor...|Good quality,work...|
| RVBP8I1R0CTZ8|in fact this is t...|This demn tablet ...|
|R1QF6RS1PDLU18|                Good|I am not sure I d...|
|R23AICGEDAJQL1|You get what you ...|After exactly 45 ...|
|R2EY3N4K9W19UP|Great for Windows...|Replaced my Intel...|
| RC9AW4HKJ016M|            One Star|IT HAS ALREADY CR...|
|R2ALWJE9N6ZBXD|   Very Disappointed|Very disappointed...|
|R2G5FPA4OX37GV|          Five Stars|Works well. I use...|
|R1IKTSEVXSIMOD|The encryption so...|The encryption so..

In [10]:
rating_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
rating_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3ARRMDEGED8RD|          5|            0|          0|   N|
| RQ28TSA020Y6J|          5|           24|         31|   N|
| RUXJRZCT6953M|          1|            2|          2|   N|
| R7EO0UO6BPB71|          1|            0|          0|   N|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|
|R31SR7REWNX7CF|          5|            0|          0|   N|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|
|R1QF6RS1PDLU18|          4|            1|          1|   N|
|R23AICGEDAJQL1|          1|            0|          0|   N|
|R2EY3N4K9W19UP|          5|            3|          4|   N|
| RC9AW4HKJ016M|          1|            0|          0|   N|
|R2ALWJE9N6ZBXD|          1|            0|          0|   N|
|R2G5FPA4OX37GV|          5|            1|          1|   N|
|R1IKTSEVXSIMOD|          5|            

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-hw22.cqgsn56zcbuj.us-west-1.rds.amazonaws.com:5432/my_data_HW22_db"
config = {"user":"root", 
          "password": "*****", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to table in RDS

reviews_df.write.jdbc(url=jdbc_url, table='reviews_df', mode=mode, properties=config)


In [ ]:
# Write vine_df to table in RDS

rating_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)